In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model, preprocessing, metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
import time

In [2]:
df = pd.read_csv('df_for_model.csv')
df = df.drop('Unnamed: 0',axis=1)

In [3]:
SiteEnergyUse_discrete = pd.qcut(df['SiteEnergyUse(kBtu)'],3, labels = False)
GHGEmission_discrete = pd.qcut(df['GHGEmissions(MetricTonsCO2e)'],3, labels = False)
df.insert(11, 'SiteEnergyUse_discrete', SiteEnergyUse_discrete)
df.insert(12, 'GHGEmission_discrete', GHGEmission_discrete)

In [4]:
df['SiteEnergyUse(kBtu)_log'] = np.log1p(df['SiteEnergyUse(kBtu)'])
df['GHGEmissions(MetricTonsCO2e)_log'] = np.log1p(df['GHGEmissions(MetricTonsCO2e)'])

In [7]:
df.columns

Index(['PrimaryPropertyType', 'Neighborhood', 'YearBuilt', 'NumberofBuildings',
       'NumberofFloors', 'PropertyGFAParking', 'PropertyGFABuilding(s)',
       'ListOfAllPropertyUseTypes', 'LargestPropertyUseType',
       'LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseType',
       'SiteEnergyUse_discrete', 'GHGEmission_discrete',
       'SecondLargestPropertyUseTypeGFA', 'ENERGYSTARScore',
       'SiteEnergyUse(kBtu)', 'GHGEmissions(MetricTonsCO2e)',
       'MedianEnergyUse', 'MedianGHGEmission', 'NumberofUseTypes',
       'Laboratory', 'Wholesale Club/Supercenter', 'Residential Care Facility',
       'Refrigerated Warehouse', 'Other/Specialty Hospital',
       'Other - Technology/Science', 'Other - Mall', 'Convention Center',
       'Strip Mall', 'Vocational School', 'Police Station', 'Hotel',
       'Food Service', 'SPS-District K-12', 'Supermarket/Grocery Store',
       'Data Center', 'Urgent Care/Clinic/Other Outpatient', 'Library',
       'Automobile Dealership', 'Senior C

In [5]:
#df=df.drop(df[df['PrimaryPropertyType']=='Hospital'].index)

In [9]:
data=df['Neighborhood'].values.reshape(-1,1)
enc = preprocessing.OneHotEncoder()
neighborhood_enc = enc.fit_transform(data).toarray()
neighborhood_enc.shape

(1811, 13)

In [10]:
data=df['PrimaryPropertyType'].values.reshape(-1,1)
enc = preprocessing.OneHotEncoder()
PropertyType_enc = enc.fit_transform(data).toarray()
PropertyType_enc.shape

(1811, 25)

In [11]:
#X_cat = np.concatenate((neighborhood_enc,PropertyType_enc),axis=1)
#X_cat.shape

(1811, 38)

In [24]:
X_cat = np.concatenate((neighborhood_enc,df[['Laboratory', 'Wholesale Club/Supercenter', 'Residential Care Facility',
       'Refrigerated Warehouse', 'Other/Specialty Hospital',
       'Other - Technology/Science', 'Other - Mall', 'Convention Center',
       'Strip Mall', 'Vocational School', 'Police Station', 'Hotel',
       'Food Service', 'SPS-District K-12', 'Supermarket/Grocery Store',
       'Data Center', 'Urgent Care/Clinic/Other Outpatient', 'Library',
       'Automobile Dealership', 'Senior Care Community',
       'Other - Restaurant/Bar', 'Convenience Store without Gas Station',
       'Parking', 'Hospital (General Medical & Surgical)',
       'Prison/Incarceration', 'Worship Facility', 'Medical Office', 'Other',
       'Bar/Nightclub', 'Other - Education', 'Self-Storage Facility',
       'Other - Public Services', 'Movie Theater', 'Swimming Pool',
       'Repair Services (Vehicle; Shoe; Locksmith; etc)', 'Single Family Home',
       'Personal Services (Health/Beauty; Dry Cleaning; etc)',
       'Energy/Power Station', 'Residence Hall/Dormitory',
       'Other - Recreation', 'Social/Meeting Hall',
       'Small- and Mid-Sized Office', 'Courthouse', 'Distribution Center',
       'Other - Services', 'Museum', 'Retail Store', 'Office',
       'College/University', 'Hospital', 'Fitness Center/Health Club/Gym',
       'Fast Food Restaurant', 'Multifamily Housing', 'Lifestyle Center',
       'Fire Station', 'Manufacturing/Industrial Plant', 'K-12 School',
       'Other - Entertainment/Public Assembly', 'Bank Branch', 'Food Sales',
       'Adult Education', 'Large Office', 'Non-Refrigerated Warehouse',
       'Outpatient Rehabilitation/Physical Therapy', 'Mixed Use Property',
       'Restaurant', 'Other - Lodging/Residential', 'Pre-school/Daycare',
       'Enclosed Mall', 'Performing Arts', 'Other - Utility',
       'Financial Office']].to_numpy()),axis=1)

In [30]:
X_quant = df[['YearBuilt', 'NumberofBuildings', 'NumberofFloors', 'PropertyGFABuilding(s)','PropertyGFAParking','LargestPropertyUseTypeGFA','SecondLargestPropertyUseTypeGFA','MedianEnergyUse','MedianGHGEmission','NumberofUseTypes']]
X_quant.values.shape

(1811, 10)

In [31]:
X_cat.shape

(1811, 85)

In [32]:
Y_energy_log = df['SiteEnergyUse(kBtu)_log']
Y_ghg_log = df['GHGEmissions(MetricTonsCO2e)_log']
Y_energy_bis = df['SiteEnergyUse(kBtu)']
Y_ghg_bis = df['GHGEmissions(MetricTonsCO2e)']

In [33]:
X_cat_train, X_cat_test, X_quant_train, X_quant_test, Y_energy_log_train, Y_energy_log_test, Y_ghg_log_train, Y_ghg_log_test, Y_energy_bis_train, Y_energy_bis_test, Y_ghg_bis_train, Y_ghg_bis_test = train_test_split(X_cat, X_quant, Y_energy_log, Y_ghg_log, Y_energy_bis, Y_ghg_bis, test_size=0.3, stratify = df['PrimaryPropertyType'], random_state = 1)

In [34]:
std = preprocessing.StandardScaler()
X_quant_train_scaled = std.fit_transform(X_quant_train)
X_quant_test_scaled = std.transform(X_quant_test)

In [35]:
X_train = np.concatenate((X_cat_train,X_quant_train_scaled),axis=1)
X_test = np.concatenate((X_cat_test,X_quant_test_scaled),axis=1)

In [36]:
# Régression linéaire sur cibles en log
lr_energy = linear_model.LinearRegression()
lr_energy.fit(X_train,Y_energy_log_train)

print('R² test sur variable log:', round(metrics.r2_score(Y_energy_log_test,lr_energy.predict(X_test)),3))
print('R² test sur variable origine:', round(metrics.r2_score(np.expm1(Y_energy_log_test),np.expm1(lr_energy.predict(X_test))),3))

print('R² train sur variable log:',round(metrics.r2_score(Y_energy_log_train,lr_energy.predict(X_train)),3))
print('R² train sur variable orig:',round(metrics.r2_score(np.expm1(Y_energy_log_train),np.expm1(lr_energy.predict(X_train))),3))

R² test sur variable log: -9.382990205844005e+21


<ipython-input-36-60a89e9fb585>:6: RuntimeWarning: overflow encountered in expm1
  print('R² test sur variable origine:', round(metrics.r2_score(np.expm1(Y_energy_log_test),np.expm1(lr_energy.predict(X_test))),3))


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [37]:
# Régression linéaire sur cibles non transformées
lr_energy_bis = linear_model.LinearRegression()
lr_energy_bis.fit(X_train,Y_energy_bis_train)

print('R² test :', round(metrics.r2_score(Y_energy_bis_test,lr_energy_bis.predict(X_test)),3))
print('R² train :',round(metrics.r2_score(Y_energy_bis_train,lr_energy_bis.predict(X_train)),3))

R² test : -4.2455173969282074e+18
R² train : 0.757


In [38]:
recap_energy = pd.DataFrame(columns = ['R²','RMSE','Prediction time','Best parameters']).rename_axis('Model for energy')
recap_ghg = pd.DataFrame(columns = ['R²','RMSE','Prediction time','Best parameters']).rename_axis('Model for GHG')

In [39]:
recap_energy_log = pd.DataFrame(columns = ['R²','RMSE','Prediction time','Best parameters']).rename_axis('Model for energy log')
recap_ghg_log = pd.DataFrame(columns = ['R²','RMSE','Prediction time','Best parameters']).rename_axis('Model for GHG log')

In [40]:
def perf(X_train,Y_train,X_test,Y_test,model,hyperparam):
    
    '''Fonction permettant la recherche des meilleurs hyperparamètres d'un modèle puis 
    l'entrainement de ce modèle avec ces meilleurs paramètres sur le jeu X_train, Y_train
    La fonction retourne les performances du modèle (R², RMSE, temps d'éxécution) et les meilleurs paramètres
    
    le paramètre 'hyperparam' de la fonction est un dictionnaire
    
    Liste des hyperparamètres selon certains modèles :
    - Régression linéaire : pas d'hyperparamètre
    - Régression Ridge : alpha
    - Regression Lasso : alpha
    - Elasticnet : alpha et l1_ratio
    - SVR : estimator__C, estimator__epsilon et estimator__kernel
    - Random Forest : 
    '''
        
    model_gs = GridSearchCV(estimator = model, param_grid=hyperparam, cv=5)      
        
    model_gs.fit(X_train, Y_train)
    start_time = time.time()
    score = round(metrics.r2_score(Y_test,model_gs.predict(X_test)),5)     
    RMSE = round(metrics.mean_squared_error(Y_test,model_gs.predict(X_test), squared = False),5)
    pred_time = round(time.time()-start_time, 6)
    best_param = model_gs.best_params_
    for k, v in best_param.items():
        if type(v)!= str :
            best_param[k] = round(v, 6)
   
    return score, RMSE, pred_time, best_param



def recap_table(X_train,Y_energy_train,Y_ghg_train,X_test,Y_energy_test,Y_ghg_test,model,hyperparam):
    score, RMSE, pred_time, best_param = perf(X_train,Y_energy_train,X_test,Y_energy_test,model,hyperparam)
    recap_energy.loc[str(model),:] = [score,RMSE,pred_time,best_param]
    
    score, RMSE, pred_time, best_param = perf(X_train,Y_ghg_train,X_test,Y_ghg_test,model,hyperparam)
    recap_ghg.loc[str(model),:] = [score,RMSE,pred_time,best_param]

In [41]:
def perf_log(X_train,Y_train,X_test,Y_test,model,hyperparam):
    
    '''Fonction permettant la recherche des meilleurs hyperparamètres d'un modèle puis 
    l'entrainement de ce modèle avec ces meilleurs paramètres sur le jeu X_train, Y_train
    La fonction retourne les performances du modèle (R², RMSE, temps d'éxécution) et les meilleurs paramètres
    
    le paramètre 'hyperparam' de la fonction est un dictionnaire
    
    Liste des hyperparamètres selon certains modèles :
    - Régression linéaire : pas d'hyperparamètre
    - Régression Ridge : alpha
    - Regression Lasso : alpha
    - Elasticnet : alpha et l1_ratio
    - SVR : estimator__C, estimator__epsilon et estimator__kernel
    - Random Forest : 
    '''
        
    model_gs = GridSearchCV(estimator = model, param_grid=hyperparam, cv=5)      
        
    model_gs.fit(X_train, Y_train)
    start_time = time.time()
    score = round(metrics.r2_score(np.expm1(Y_test),np.expm1(model_gs.predict(X_test))),5)     
    RMSE = round(metrics.mean_squared_error(np.expm1(Y_test),np.expm1(model_gs.predict(X_test)), squared = False),5)
    pred_time = round(time.time()-start_time, 6)
    best_param = model_gs.best_params_
    for k, v in best_param.items():
        if type(v)!= str :
            best_param[k] = round(v, 6)
   
    return score,RMSE, pred_time, best_param



def recap_table_log(X_train,Y_energy_train,Y_ghg_train,X_test,Y_energy_test,Y_ghg_test,model,hyperparam):
    score, RMSE, pred_time, best_param = perf_log(X_train,Y_energy_train,X_test,Y_energy_test,model,hyperparam)
    recap_energy_log.loc[str(model),:] = [score,RMSE,pred_time,best_param]
    
    score, RMSE, pred_time, best_param = perf_log(X_train,Y_ghg_train,X_test,Y_ghg_test,model,hyperparam)
    recap_ghg_log.loc[str(model),:] = [score,RMSE,pred_time,best_param]

In [42]:
LinearReg = linear_model.LinearRegression();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,LinearReg,{})

In [43]:
ridge = Ridge();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,ridge,{'alpha': np.logspace(-4,2,100)})

In [44]:
lasso = Lasso();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,lasso,{'alpha': np.logspace(-4,2,100)})

C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.234477753332822e+16, tolerance: 56314712393550.78
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.1083226380439896e+16, tolerance: 35302160439502.13
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.606723710751394e+16, tolerance: 63166560028773.97
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:5

In [45]:
svr = SVR(kernel = 'linear');
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,svr,{'C' : range(1,5),
                                       'epsilon' : np.linspace(0.1,1,10)})

In [46]:
svr = SVR();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test,svr,{'C' : range(1,5),
                                        'epsilon' : np.linspace(0.1,1,10), 
                                       'kernel' : ['rbf','poly','sigmoid']})

In [65]:
rfr = RandomForestRegressor();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test, rfr, {'n_estimators' : [300] ,'min_samples_split' : [2, 5, 10], 
                                             'min_samples_leaf' : [1, 2, 4], 
                                             'max_depth': range(5,16,1)})

In [ ]:
gb = LGBMRegressor();
recap_table(X_train,Y_energy_bis_train,Y_ghg_bis_train,X_test,Y_energy_bis_test,Y_ghg_bis_test, gb, {'n_estimators': [150],'max_depth':range(2,16), 
                                                                                                     'learning_rate' : [0.0001,0.001,0.01,0.1], 
                                                                                                     'boosting_type' : ['gbdt','goss', 'dart']})

In [66]:
recap_energy

,R²,RMSE,Prediction time,Best parameters
Model for energy,,,,
LinearRegression(),-4.24552e+18,8.4137e+16,0.001001,{}
Ridge(),0.8963,1.31494e+07,0.001,{'alpha': 0.284804}
Lasso(),0.90492,1.25909e+07,0.002,{'alpha': 100.0}
SVR(kernel='linear'),-0.02273,4.12955e+07,0.088513,"{'C': 4, 'epsilon': 0.1}"
SVR(),0.18213,3.69287e+07,0.110008,"{'C': 4, 'epsilon': 0.1, 'kernel': 'poly'}"
RandomForestRegressor(),0.32839,3.34643e+07,0.022002,"{'max_depth': 7, 'min_samples_leaf': 4, 'min_s..."
LGBMRegressor(),0.22356,3.59812e+07,0.010002,"{'boosting_type': 'gbdt', 'learning_rate': 0.0..."


In [67]:
recap_ghg

,R²,RMSE,Prediction time,Best parameters
Model for GHG,,,,
LinearRegression(),-9.22362e+19,7.3468e+12,0.001998,{}
Ridge(),0.57004,501.604,0.000998,{'alpha': 0.162975}
Lasso(),0.59019,489.708,0.001002,{'alpha': 1.747528}
SVR(kernel='linear'),0.61988,471.639,0.092008,"{'C': 4, 'epsilon': 1.0}"
SVR(),0.0706,737.477,0.150012,"{'C': 4, 'epsilon': 0.6, 'kernel': 'sigmoid'}"
RandomForestRegressor(),0.70377,416.351,0.035,"{'max_depth': 13, 'min_samples_leaf': 1, 'min_..."
LGBMRegressor(),0.52716,526.021,0.013001,"{'boosting_type': 'dart', 'learning_rate': 0.1..."


In [68]:
recap_energy.iloc[-2,-1]

{'max_depth': 7,
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'n_estimators': 100}

In [69]:
recap_ghg.iloc[-2,-1]

{'max_depth': 13,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 100}

In [49]:
LinearReg = linear_model.LinearRegression();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,LinearReg,{})

<ipython-input-41-b1f33cbbc322>:22: RuntimeWarning: overflow encountered in expm1
  score = round(metrics.r2_score(np.expm1(Y_test),np.expm1(model_gs.predict(X_test))),5)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [50]:
ridge = Ridge();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,ridge,{'alpha': np.logspace(-4,2,100)})

In [51]:
lasso = Lasso();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,lasso,{'alpha': np.logspace(-4,2,100)})

C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 164.5676508702308, tolerance: 0.18956612146077104
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 122.61973723664374, tolerance: 0.18956612146077104
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 70.68054473398945, tolerance: 0.18956612146077104
  model = cd_fast.enet_coordinate_descent(
C:\Users\BENAVEN\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: Co

In [52]:
svr = SVR(kernel = 'linear');
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,svr,{'C' : range(1,5),
                                       'epsilon' : np.linspace(0.1,1,10)})

In [53]:
svr = SVR();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test,svr,{'C' : range(1,5),
                                        'epsilon' : np.linspace(0.1,1,10), 
                                       'kernel' : ['rbf','poly','sigmoid']})

In [54]:
rfr = RandomForestRegressor();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test, rfr, {'min_samples_split' : [2, 5, 10], 
                                             'min_samples_leaf' : [1, 2, 4], 
                                             'max_depth': range(5,16,1)})

In [55]:
gb = LGBMRegressor();
recap_table_log(X_train,Y_energy_log_train,Y_ghg_log_train,X_test,Y_energy_log_test,Y_ghg_log_test, gb, {'n_estimators': [150],'max_depth':range(2,16), 
                                                                                                     'learning_rate' : [0.0001,0.001,0.01,0.1], 
                                                                                                     'boosting_type' : ['gbdt','goss', 'dart']})

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^m

In [56]:
recap_energy_log

,R²,RMSE,Prediction time,Best parameters
Model for energy log,,,,
Ridge(),-4.57473e+23,2.76188e+19,0.001999,{'alpha': 3.053856}
Lasso(),-5.20824e+23,2.94691e+19,0.001001,{'alpha': 0.001417}
SVR(kernel='linear'),-5.68873e+30,9.73934e+22,0.061017,"{'C': 1, 'epsilon': 0.3}"
SVR(),0.1259,3.81771e+07,0.123009,"{'C': 3, 'epsilon': 0.1, 'kernel': 'rbf'}"
RandomForestRegressor(),0.30654,3.40041e+07,0.026002,"{'max_depth': 11, 'min_samples_leaf': 1, 'min_..."
LGBMRegressor(),0.32213,3.36198e+07,0.006001,"{'boosting_type': 'gbdt', 'learning_rate': 0.1..."


In [57]:
recap_ghg_log

,R²,RMSE,Prediction time,Best parameters
Model for GHG log,,,,
Ridge(),-1.52087e+23,2.98327e+14,0.000998,{'alpha': 4.641589}
Lasso(),-3.51195e+23,4.53337e+14,0.001002,{'alpha': 0.003275}
SVR(kernel='linear'),-4.16005e+24,1.56026e+15,0.104009,"{'C': 1, 'epsilon': 0.2}"
SVR(),0.18109,692.254,0.104007,"{'C': 3, 'epsilon': 0.3, 'kernel': 'rbf'}"
RandomForestRegressor(),0.60104,483.183,0.035004,"{'max_depth': 15, 'min_samples_leaf': 1, 'min_..."
LGBMRegressor(),0.53358,522.441,0.010001,"{'boosting_type': 'gbdt', 'learning_rate': 0.1..."


In [144]:
recap_energy_log.iloc[-1,-1]

{'boosting_type': 'gbdt',
 'learning_rate': 0.065,
 'max_depth': 7,
 'n_estimators': 120,
 'num_leaves': 30}